In [1]:
import numpy as np
import math
import csv
import matplotlib.pyplot as plt
import seaborn as sns
import time
import random
import pandas as pd
from pyomo.opt import SolverFactory
from pyomo.core import Var
import pyomo.environ as en
%matplotlib inline

In [2]:
# input data
testData = pd.read_csv('testdata_final.csv', parse_dates=['timestamp'], index_col='timestamp')

In [3]:
# define input data
load = testData['actual_consumption'].values
# PV_unit = testData['pv_c'].fillna(0).values/1000      # traditional PV model
sellPrice = testData['price_sell'].values
buyPrice = testData['price_buy'].values
heat_load = testData['heat_load'].values
cool_load = testData['cool_load'].values
price_gas1 = testData['gas'].values/1000
price_gas = price_gas1*35.3147248277 
# The unit of natural gas price is dollars per thousand cubic feet
# convert into dollers per cubic meter 

In [4]:
PV_unit = testData['actual_pv'].fillna(0)
PV_unit[PV_unit<0] = 0
PV_unit = PV_unit.values/1000    # The pv output unit in the data is W, converted to kW

In [5]:
priceDict1 = dict(enumerate(sellPrice))
priceDict2 = dict(enumerate(buyPrice))
priceDict3 = dict(enumerate(price_gas))

In [6]:
LoadDict = dict(enumerate(load))
PVDict = dict(enumerate(PV_unit))
HeatDict = dict(enumerate(heat_load))
CoolDict = dict(enumerate(cool_load))

In [7]:
m = en.ConcreteModel()

In [8]:
m.Time = en.RangeSet(0,len(load)-1)

In [9]:
m.priceSell = en.Param(m.Time, initialize=priceDict1)
m.priceBuy = en.Param(m.Time, initialize=priceDict2)
m.priceGas = en.Param(m.Time, initialize=priceDict3)
m.Load = en.Param(m.Time, initialize=LoadDict)
m.PV_unit = en.Param(m.Time, initialize=PVDict)
m.Heat = en.Param(m.Time,initialize=HeatDict)
m.Cool = en.Param(m.Time,initialize=CoolDict)

In [10]:
m.P_buy = en.Var(m.Time, within=en.Reals)
m.P_sell = en.Var(m.Time, within=en.Reals)

In [11]:
# Define TES parameters
m.TSS_Capacity = en.Param(initialize=10)
m.TSS_Power = en.Param(initialize=10)
m.TSS_Charge_Efficiency = en.Param(initialize=0.95)
m.TSS_Discharge_Efficiency = en.Param(initialize=0.95)
m.TSS_rate = en.Param(initialize=0.04)

In [12]:
# Define heat storage parameters
m.Bool_char_TSS = en.Var(m.Time,within=en.Binary)
m.Bool_dis_TSS = en.Var(m.Time,within=en.Binary,initialize=0)
m.Energy_SOC_TSS = en.Var(m.Time,initialize=0)
m.TSS_P_char = en.Var(m.Time,initialize=0)
m.TSS_P_dis = en.Var(m.Time,initialize=0)
m.n_TSS = en.Var([1],within=en.NonNegativeIntegers)

In [13]:
bigM = 10000000
m.z1_TSS = en.Var(m.Time)
m.z2_TSS = en.Var(m.Time)

In [14]:
# Introduce auxiliary variable 1
def Bool_char_TSS_rule_1(m,i):
    return m.z1_TSS[i] <= bigM*m.Bool_char_TSS[i]
m.TSS_char_cons1 = en.Constraint(m.Time, rule=Bool_char_TSS_rule_1)

def Bool_char_TSS_rule_2(m,i):
    return -bigM*m.Bool_char_TSS[i] <= m.z1_TSS[i]
m.TSS_char_cons2 = en.Constraint(m.Time, rule=Bool_char_TSS_rule_2)

def Bool_char_TSS_rule_3(m,i):
    return m.z1_TSS[i]-m.n_TSS[1] <= bigM*(1-m.Bool_char_TSS[i])
m.TSS_char_cons3 = en.Constraint(m.Time, rule=Bool_char_TSS_rule_3)

def Bool_char_TSS_rule_4(m,i):
    return -bigM*(1-m.Bool_char_TSS[i]) <= m.z1_TSS[1]-m.n_TSS[1]
m.TSS_char_cons4 = en.Constraint(m.Time, rule=Bool_char_TSS_rule_4)

In [15]:
# Introduce auxiliary variable 1
def Bool_dis_TSS_rule_1(m,i):
    return m.z2_TSS[i] <= bigM*m.Bool_dis_TSS[i]
m.TSS_dis_cons1 = en.Constraint(m.Time,rule=Bool_dis_TSS_rule_1)

def Bool_dis_TSS_rule_2(m,i):
    return -bigM*m.Bool_dis_TSS[i] <= m.z2_TSS[i]
m.TSS_dis_cons2 = en.Constraint(m.Time,rule=Bool_dis_TSS_rule_2)

def Bool_dis_TSS_rule_3(m,i):
    return m.z2_TSS[i]-m.n_TSS[1] <= bigM*(1-m.Bool_dis_TSS[i])
m.TSS_dis_cons3 = en.Constraint(m.Time,rule=Bool_dis_TSS_rule_3)

def Bool_dis_TSS_rule_4(m,i):
    return -bigM*(1-m.Bool_dis_TSS[i]) <= m.z2_TSS[i]-m.n_TSS[1]
m.TSS_dis_cons4 = en.Constraint(m.Time,rule=Bool_dis_TSS_rule_4)

In [16]:
# The thermal energy storage power cannot exceed the upper and lower limits
# Nonlinear transformation： m.z2_ESS = m.Bool_char_TSS * m.n_TSS
m.ChargingLimit = en.Param(initialize=m.TSS_Power)

def TSS_char_power_rule_1(m,i):
    return m.TSS_P_char[i] <= m.z1_TSS[i]*m.ChargingLimit
m.TSS_char_power_cons1 = en.Constraint(m.Time,rule=TSS_char_power_rule_1)

def TSS_char_power_rule_2(m,i):
    return 0 <= m.TSS_P_char[i]
m.TSS_char_power_cons2 = en.Constraint(m.Time,rule=TSS_char_power_rule_2)

In [17]:
# The heating power of thermal storage cannot exceed the upper and lower limits
# Nonlinear transformation： m.z2_TSS = m.Bool_dis_TSS * m.n_TSS
m.DischargingLimit = en.Param(initialize=m.TSS_Power)

def TSS_dis_power_rule_1(m,i):
    return m.TSS_P_dis[i] <= m.z2_TSS[i]*m.DischargingLimit
m.TSS_dis_power_cons1 = en.Constraint(m.Time, rule=TSS_dis_power_rule_1)

def TSS_dis_power_rule_2(m,i):
    return 0 <= m.TSS_P_dis[i]
m.TSS_dis_power_cons2 = en.Constraint(m.Time,rule=TSS_dis_power_rule_2)

In [18]:
# TES is not allowed to store and release energy at the same time
def TSS_char_dis(m,i):
    return m.Bool_char_TSS[i]+m.Bool_dis_TSS[i] <= 1
m.TSS_char_dis_cons = en.Constraint(m.Time,rule=TSS_char_dis)

In [19]:
# Heat storage energy
def Energy_SOC_TSS_rule(m,t):
    if t==0:
        return m.Energy_SOC_TSS[t] == 0.2*m.TSS_Capacity*m.n_TSS[1]
    else:
        return m.Energy_SOC_TSS[t] == m.Energy_SOC_TSS[t-1]*(1-m.TSS_rate)+m.TSS_Charge_Efficiency*m.TSS_P_char[t]-m.TSS_P_dis[t]/m.TSS_Discharge_Efficiency                    
m.TSS_Energy_SOC_cons = en.Constraint(m.Time,rule=Energy_SOC_TSS_rule)

def Energy_TSS_rule_1(m,t):
    return m.Energy_SOC_TSS[t] <= 0.9*m.TSS_Capacity*m.n_TSS[1]
m.TSS_Energy_cons1 = en.Constraint(m.Time,rule=Energy_TSS_rule_1)

def Energy_TSS_rule_2(m,t):
    return 0.2*m.TSS_Capacity*m.n_TSS[1] <= m.Energy_SOC_TSS[t]
m.TSS_Energy_cons2 = en.Constraint(m.Time,rule=Energy_TSS_rule_2)

In [20]:
# Number of heat storage installations
def TSS_num_rule(m):
    return m.n_TSS[1] <= 50000
m.TSS_num_cons = en.Constraint(m.Time,rule=TSS_num_rule)

In [21]:
# Define energy storage battery parameters
m.Batt_Capacity = en.Param(initialize=1)
m.Batt_Power = en.Param(initialize=0.075)
m.Batt_Charge_Efficiency = en.Param(initialize=0.95)
m.Batt_Discharge_Efficiency = en.Param(initialize=0.95)
m.Batt_rate = en.Param(initialize=0.04)

In [22]:
# Define energy storage battery parameters
m.Bool_char = en.Var(m.Time,within=en.Binary)
m.Bool_dis = en.Var(m.Time,within=en.Binary,initialize=0)
m.Energy_SOC = en.Var(m.Time,initialize=0)
m.Batt_P_char = en.Var(m.Time,initialize=0)
m.Batt_P_dis = en.Var(m.Time,initialize=0)

m.n_ESS = en.Var([1],within=en.NonNegativeIntegers)

In [23]:
# bigM 
bigM = 10000000
m.z1_Batt = en.Var(m.Time)
m.z2_Batt = en.Var(m.Time)

In [24]:
# Introduce auxiliary variable 1
def Bool_char_rule_1(m,i):
    return( m.z1_Batt[i] <= bigM*m.Bool_char[i] )
m.Batt_char1 = en.Constraint(m.Time,rule=Bool_char_rule_1)

def Bool_char_rule_2(m,i):
    return( -bigM*m.Bool_char[i] <= m.z1_Batt[i] )
m.Batt_char2 = en.Constraint(m.Time,rule=Bool_char_rule_2)

def Bool_char_rule_3(m,i):
    return( m.z1_Batt[i]-m.n_ESS[1] <= bigM*(1-m.Bool_char[i]) )
m.Batt_char3 = en.Constraint(m.Time,rule=Bool_char_rule_3)

def Bool_char_rule_4(m,i):
    return( -bigM*(1-m.Bool_char[i]) <= m.z1_Batt[i]-m.n_ESS[1] )
m.Batt_char4 = en.Constraint(m.Time, rule=Bool_char_rule_4)

In [25]:
# Introduce auxiliary variable 2
def Bool_dis_rule_1(m,i):
    return( m.z2_Batt[i] <= bigM*m.Bool_dis[i] )
m.Batt_dis1 = en.Constraint(m.Time,rule=Bool_dis_rule_1)

def Bool_dis_rule_2(m,i):
    return( -bigM*m.Bool_dis[i] <= m.z2_Batt[i] )
m.Batt_dis2 = en.Constraint(m.Time,rule=Bool_dis_rule_2)

def Bool_dis_rule_3(m,i):
    return( m.z2_Batt[i]-m.n_ESS[1] <= bigM*(1-m.Bool_dis[i]) )
m.Batt_dis3 = en.Constraint(m.Time,rule=Bool_dis_rule_3)

def Bool_dis_rule_4(m,i):
    return( -bigM*(1-m.Bool_dis[i]) <= m.z2_Batt[i]-m.n_ESS[1] )
m.Batt_dis4 = en.Constraint(m.Time,rule=Bool_dis_rule_4)

In [26]:
# The charging power of the electric storage cannot exceed the upper and lower limits
# Nonlinear transformation： m.z1_ESS = m.Bool_char * m.n_ESS
m.ChargingLimit_Batt = en.Param(initialize=m.Batt_Power)

def Batt_char_power_rule_1(m,i):
    return( m.Batt_P_char[i] <= m.z1_Batt[i]*m.ChargingLimit_Batt )
m.Batt_char_power1 = en.Constraint(m.Time,rule=Batt_char_power_rule_1)

def Batt_char_power_rule_2(m,i):
    return( 0 <= m.Batt_P_char[i] )
m.Batt_char_power2 = en.Constraint(m.Time, rule=Batt_char_power_rule_2)

In [27]:
# The discharge power of the electric energy storage cannot exceed the upper and lower limits，
# Nonlinear transformation： m.z2_ESS = m.Bool_dis * m.n_ESS
m.DischargingLimit_Batt = en.Param(initialize=m.Batt_Power)

def Batt_dis_power_rule_1(m,i):
    return( m.Batt_P_dis[i] <= m.z2_Batt[i]*m.DischargingLimit_Batt )
m.Batt_dis_power1 = en.Constraint(m.Time,rule=Batt_dis_power_rule_1)

def Batt_dis_power_rule_2(m,i):
    return( 0 <= m.Batt_P_dis[i] )
m.Batt_dis_power2 = en.Constraint(m.Time, rule=Batt_dis_power_rule_2)

In [28]:
# EES is not allowed to store and release energy at the same time
def Batt_char_dis(m,i):
    return (m.Bool_char[i]+m.Bool_dis[i] <= 1)
m.Batt_char_dis=en.Constraint(m.Time,rule=Batt_char_dis)

In [29]:
# Energy of EES
def Energy_SOC_rule(m,t):
    if t==0:
        return ( m.Energy_SOC[t] == 0.2*m.Batt_Capacity*m.n_ESS[1] )
    else:
        return ( m.Energy_SOC[t] == m.Energy_SOC[t-1]*(1-m.Batt_rate)+m.Batt_Charge_Efficiency*m.Batt_P_char[t]-m.Batt_P_dis[t]/m.Batt_Discharge_Efficiency )
m.Batt_Energy_SOC = en.Constraint(m.Time,rule=Energy_SOC_rule)    

def Energy_ESS_rule_1(m,t):
    return( m.Energy_SOC[t] <= 0.9*m.Batt_Capacity*m.n_ESS[1] )
m.Batt_Energy_ESS1 = en.Constraint(m.Time,rule=Energy_ESS_rule_1)

def Energy_ESS_rule_2(m,t):
    return( 0.2*m.Batt_Capacity*m.n_ESS[1] <= m.Energy_SOC[t] )
m.Batt_Energy_ESS2 = en.Constraint(m.Time,rule=Energy_ESS_rule_2)

In [30]:
# Number of electric energy storage installations
def ESS_num_rule(m):
    return m.n_ESS[1] <= 500000
m.ESS_num_cons = en.Constraint(m.Time, rule=ESS_num_rule)

In [31]:
# electrical chillers
m.n_EC = en.Var([1],within=en.NonNegativeIntegers)
m.P_EC = en.Var(m.Time,within=en.Reals)
m.Q_EC = en.Var(m.Time,within=en.Reals)
m.EC_COP = en.Param(initialize=4)
m.P_EC_Max = en.Param(initialize=200)

In [32]:
# Number of electric chillers installed
def EC_num_rule(m):
    return m.n_EC[1] <= 10000
m.EC_num_cons = en.Constraint(rule=EC_num_rule)

In [33]:
# Output constraint of electric chiller
def P_EC_rule1(m,i):
    return m.P_EC[i] <= m.P_EC_Max*m.n_EC[1]
m.P_EC_cons1 = en.Constraint(m.Time,rule=P_EC_rule1)

In [34]:
# Output constraint of electric chiller
def P_EC_rule2(m,i):
    return 0 <= m.P_EC[i]
m.P_EC_cons2 = en.Constraint(m.Time,rule=P_EC_rule2)

In [35]:
# Electrical chiller is converted to cold energy
def EC_Ele_to_cool_rule(m,i):
    return m.Q_EC[i] == m.P_EC[i]*m.EC_COP
m.EC_ELE_TO_COOL_cons = en.Constraint(m.Time,rule=EC_Ele_to_cool_rule)

In [36]:
# Gas turbine
m.n_GT = en.Var([1],within=en.NonNegativeIntegers)
m.P_GT = en.Var(m.Time,within=en.Reals)
m.GT_Max = en.Param(initialize=400)
m.GT_to_Electricity_Efficient = en.Param(initialize=0.3657)

In [37]:
# Calorific value of natural gas
m.H_gas = en.Param(initialize=9.97)
# Gas turbine consumption
m.Vgas_GT = en.Var(m.Time,within=en.Reals)

In [38]:
# CHP parameters
m.n_CHP = en.Var([1],within=en.NonNegativeIntegers)
m.P_CHP = en.Var(m.Time,within=en.Reals)
m.Q_CHP = en.Var(m.Time,within=en.Reals)
m.CHP_to_Electricity_Efficiency = en.Param(initialize=0.3)
m.CHP_to_Heat_Efficiency = en.Param(initialize=0.45)
m.P_CHP_Max = en.Param(initialize=300)
m.Q_CHP_Max = en.Param(initialize=450)
m.Vgas_CHP = en.Var(m.Time,within=en.Reals)
m.Vgas_CHP_Q = en.Var(m.Time,within=en.Reals)

In [39]:
# AC paramters
m.n_AC = en.Var([1],within=en.NonNegativeIntegers)
m.Q_AC = en.Var(m.Time,within=en.Reals)
m.Vgas_AC = en.Var(m.Time,within=en.Reals)
m.AC_COP = en.Param(initialize=1.3)
m.Q_AC_Max = en.Param(initialize=100)

In [40]:
# Number of AC
def AC_num_rule(m,i):
    return m.n_AC[1] <= 100
m.AC_num_cons = en.Constraint(rule=AC_num_rule)

In [42]:
# AC absorbs the residual heat of CHP
def AC_gas_rule(m,i):
    return m.Vgas_CHP[i] == m.Vgas_CHP_Q[i] + m.Vgas_AC[i]
m.AC_gas_cons = en.Constraint(m.Time,rule=AC_gas_rule)

In [43]:
# AC model
def AC_heat_to_cool_rule(m,i):
    return m.Q_AC[i] == m.AC_COP*m.H_gas*m.Vgas_AC[i]
m.AC_heat_to_cool_cons = en.Constraint(m.Time,rule=AC_heat_to_cool_rule)

In [44]:
# AC output constraint
def AC_rate_rule1(m,i):
    return m.Q_AC[i] <= m.Q_AC_Max*m.n_AC[1]
m.AC_rate_cons1 = en.Constraint(m.Time,rule=AC_rate_rule1)

In [45]:
# AC output constraint
def AC_rate_rule2(m,i):
    return 0 <= m.Q_AC[i]
m.AC_rate_cons2 = en.Constraint(m.Time,rule=AC_rate_rule2)

In [46]:
# define PV deployment number
m.n_PV = en.Var([1],within=en.NonNegativeIntegers)

In [47]:
# define PV output power
m.P_PV = en.Var(m.Time, within=en.Reals)

In [48]:
# Number of photovoltaic installations
def PV_num_rule(m):
    return m.n_PV[1] <= 200000
m.PV_num_cons = en.Constraint(m.Time, rule=PV_num_rule)

In [49]:
# Number of gas turbine installations
def GT_num_rule(m):
    return m.n_GT[1] <= 40
m.GT_num_cons = en.Constraint(rule=GT_num_rule)

In [50]:
# Number of CHP installations
def CHP_num_rule(m):
    return m.n_CHP[1] <= 40
m.CHP_num_cons = en.Constraint(rule=CHP_num_rule)

In [51]:
# # Number of CHPS, not installed
# def CHP_num_rule(m):
#     return m.n_CHP[1] == 0
# m.CHP_num_cons = en.Constraint(rule=CHP_num_rule)

In [52]:
# CHP power output  
def CHP_electricity_rule(m,i):
    return m.P_CHP[i] == m.CHP_to_Electricity_Efficiency*m.Vgas_CHP[i]*m.H_gas
m.CHP_ele_cons = en.Constraint(m.Time,rule=CHP_electricity_rule)

In [53]:
# CHP heating output  
def CHP_heat_rule(m,i):
    return m.Q_CHP[i] == m.CHP_to_Heat_Efficiency*m.Vgas_CHP_Q[i]*m.H_gas
m.CHP_heat_cons = en.Constraint(m.Time,rule=CHP_heat_rule)

In [54]:
# CHP power constraints
def P_CHP_rule(m,i):
    return m.P_CHP[i] <= m.P_CHP_Max*m.n_CHP[1]
m.P_CHP_cons = en.Constraint(m.Time,rule=P_CHP_rule)

In [55]:
# CHP power constraints
def P_CHP_rule2(m,i):
    return 0 <= m.P_CHP[i]
m.P_CHP_cons2 = en.Constraint(m.Time,rule=P_CHP_rule2)

In [56]:
# CHP heating power constraints
def Q_CHP_rule(m,i):
    return m.Q_CHP[i] <= m.Q_CHP_Max*m.n_CHP[1]
m.Q_CHP_cons = en.Constraint(m.Time,rule=Q_CHP_rule)

In [57]:
# CHP heating power constraints
def Q_CHP_rule2(m,i):
    return 0 <= m.Q_CHP[i]
m.Q_CHP_cons2 = en.Constraint(m.Time,rule=Q_CHP_rule2)

In [58]:
# GT model
def GT_define_rule(m,i):
    return m.Vgas_GT[i]*m.H_gas*m.GT_to_Electricity_Efficient == m.P_GT[i]
m.GT_define_cons = en.Constraint(m.Time,rule=GT_define_rule)

In [59]:
# Gas turbine power rating
def GT_rate_rule(m,i):
    return m.P_GT[i] <= m.n_GT[1]*m.GT_Max
m.GT_rate_cons = en.Constraint(m.Time,rule=GT_rate_rule)

In [60]:
# Gas turbine power rating
def GT_rate_rule2(m,i):
    return 0 <= m.P_GT[i]
m.GT_rate2_cons = en.Constraint(m.Time,rule=GT_rate_rule2)

In [61]:
# electricity purchase
def P_buy_rule2(m,i):
    return 0 <= m.P_buy[i]
m.P_buy_cons2 = en.Constraint(m.Time, rule=P_buy_rule2)

In [62]:
# Electricity trading constraints (sell) maximum
def P_sell_rule1(m,i):
   # return m.P_sell[i] <= m.PV_unit[i] * m.n_PV[1]
    return m.P_sell[i] <= 500
m.P_sell_cons1 = en.Constraint(m.Time, rule=P_sell_rule1)

In [63]:
# Electricity trading constraints (sell)  minimum
def P_sell_rule2(m,i):
    return 0 <= m.P_sell[i]
m.P_sell_cons2 = en.Constraint(m.Time, rule=P_sell_rule2)

In [64]:
# PV power
def P_PV_rule1(m,i):
    return m.P_PV[i] == m.PV_unit[i] * m.n_PV[1]
m.P_PV_cons1 = en.Constraint(m.Time, rule=P_PV_rule1)

In [65]:
# PV output constrains minimum 
def P_PV_rule2(m,i):
    return 0 <= m.P_PV[i]
m.P_PV_cons2 = en.Constraint(m.Time, rule=P_PV_rule2)

In [66]:
# HP parameters
m.P_HP = en.Var(m.Time, within=en.Reals)
m.n_HP = en.Var([1],within=en.NonNegativeIntegers)
m.HP_COP = en.Param(initialize=2)
m.P_HP_Max = en.Param(initialize=400)

In [67]:
# heat pump power constraints maximum
def P_HP_rule1(m,i):
    return( m.P_HP[i] <= m.n_HP[1]*400 )
m.P_HP_cons1 = en.Constraint(m.Time, rule=P_HP_rule1)

In [68]:
# heat pump power constraints minimax
def P_HP_rule2(m,i):
    return( 0 <= m.P_HP[i] )
m.P_HP_cons2 = en.Constraint(m.Time, rule=P_HP_rule2)

In [69]:
# heat pump investment constraint
def num_HP_rule(m,i):
    return( m.n_HP[1] <= 30000 )
m.num_HP_cons = en.Constraint(rule=num_HP_rule)

In [70]:
#def P_buy_rule1(m,i):
#    return m.P_buy[i] <= m.Load[i]
#m.P_buy_cons1 = en.Constraint(m.Time, rule=P_buy_rule1)

In [71]:
# electricity trading constraint
def P_buy_rule1(m,i):
    return m.P_buy[i] <= 500
m.P_buy_cons1 = en.Constraint(m.Time, rule=P_buy_rule1)

In [72]:
# 
def H_balance_rule(m,i):
    return( m.P_HP[i]*2 + m.Q_CHP[i] + m.TSS_P_dis[i] == m.Heat[i] + m.TSS_P_char[i] )
m.H_balance_cons = en.Constraint(m.Time,rule=H_balance_rule)

In [73]:
# energy balance
def E_balance_rule(m,i):
    return m.Load[i] + m.P_sell[i] + m.P_HP[i] + m.P_EC[i] + m.Batt_P_char[i] == m.P_PV[i] + m.P_buy[i] + m.P_GT[i] + m.P_CHP[i] + m.Batt_P_dis[i]    
m.E_balance_cons = en.Constraint(m.Time, rule=E_balance_rule)

In [74]:
# cooling balance
def C_balance_rule(m,i):
    return m.Q_EC[i] + m.Q_AC[i] == m.Cool[i]
m.C_balance_cons = en.Constraint(m.Time,rule=C_balance_rule)

In [75]:
# discount rate
r = 0.08
R25 = (r*((1+r)**25))/( ((1+r)**25) - 1 ) 
R20 = (r*((1+r)**20))/( ((1+r)**20) - 1 ) 
R10 = (r*((1+r)**10))/( ((1+r)**10) - 1 ) 

In [76]:
# + 0.03*(158+198)*m.n_PV[1] + 0.03*70572*m.n_HP[1] + 0.03*203808*m.n_GT[1] + 0.03*505766*m.n_CHP[1] + 0.03*36438*m.n_EC[1] + 0.03*27483*m.n_AC[1] + 0.03*444*m.n_ESS[1] + 0.03*1400*m.n_TSS[1]

In [77]:
# objective function
def Obj_fn(m):
    return ( sum( (m.priceBuy[i]*m.P_buy[i]) - (m.priceSell[i]*m.P_sell[i]) + (1*5*0.00089*m.P_buy[i]) + (1*5*0.0001862*m.H_gas*(m.Vgas_GT[i]+m.Vgas_CHP[i])) + (m.priceGas[i]*(m.Vgas_GT[i]+m.Vgas_CHP[i])) for i in m.Time ) + R25*(198)*m.n_PV[1] + R25*70572*m.n_HP[1] + R25*203808*m.n_GT[1] + R25*505766*m.n_CHP[1] + R25*36438*m.n_EC[1] + R25*27483*m.n_AC[1] + R10*444*m.n_ESS[1] + R20*1400*m.n_TSS[1] + 0.03*198*m.n_PV[1] + 0.03*70572*m.n_HP[1] + 0.03*203808*m.n_GT[1] + 0.03*505766*m.n_CHP[1] + 0.03*36438*m.n_EC[1] + 0.03*27483*m.n_AC[1] + 0.03*444*m.n_ESS[1] + 0.03*1400*m.n_TSS[1] )                
m.total_cost = en.Objective(rule=Obj_fn, sense=en.minimize)

In [78]:
# Consider carbon dioxide emissions
#def Obj_fn(m):
 #   return ( sum( (m.priceBuy[i]*m.P_buy[i]) - (m.priceSell[i]*m.P_sell[i]) + (5*0.00089*m.P_buy[i]) + (5*0.0001862*m.H_gas*(m.Vgas_GT[i]+m.Vgas_CHP[i])) + (m.priceGas[i]*(m.Vgas_GT[i]+m.Vgas_CHP[i])) for i in m.Time ) + R25*(158+198)*m.n_PV[1] + R25*70572*m.n_HP[1] + R25*203808*m.n_GT[1] + R25*505766*m.n_CHP[1] + R25*36438*m.n_EC[1] + R25*27483*m.n_AC[1] + R10*444*m.n_ESS[1] + R20*1400*m.n_TSS[1] )                
#m.total_cost = en.Objective(rule=Obj_fn, sense=en.minimize)

In [79]:
# Economics alone
#def Obj_fn(m):
#    return ( sum( (m.priceBuy[i]*m.P_buy[i]) - (m.priceSell[i]*m.P_sell[i]) + (m.priceGas[i]*(m.Vgas_GT[i]+m.Vgas_CHP[i])) for i in m.Time ) + R25*(158+198)*m.n_PV[1] + R25*70572*m.n_HP[1] + R25*203808*m.n_GT[1] + R25*505766*m.n_CHP[1] + R25*36438*m.n_EC[1] + R25*27483*m.n_AC[1] + R10*444*m.n_ESS[1] + R20*1400*m.n_TSS[1]   )
#m.total_cost = en.Objective(rule=Obj_fn, sense=en.minimize)

In [80]:
#def Obj_fn(m):
#    return ( sum( (m.priceBuy[i]*m.P_buy[i]) - (m.priceSell[i]*m.P_sell[i]) + (m.priceGas[i]*(m.Vgas_GT[i]+m.Vgas_CHP[i])) for i in m.Time ) + R25*70572*m.n_HP[1] + R25*203808*m.n_GT[1] + R25*505766*m.n_CHP[1] + R25*36438*m.n_EC[1] + R25*27483*m.n_AC[1] + R10*444*m.n_ESS[1] + R20*1400*m.n_TSS[1]   )
#m.total_cost = en.Objective(rule=Obj_fn, sense=en.minimize)

In [81]:
opt = SolverFactory("cplex", executable="C:/Program Files/IBM/ILOG/CPLEX_Studio128/cplex/bin/x64_win64/cplex")

In [82]:
results = opt.solve(m)

In [83]:
print(results)


Problem: 
- Name: tmp5yd4duk3
  Lower bound: 1459517.7317
  Upper bound: 1459634.4360115612
  Number of objectives: 1
  Number of constraints: 543126
  Number of variables: 245289
  Number of nonzeros: 1116871
  Sense: minimize
Solver: 
- Status: ok
  User time: 2281.72
  Termination condition: optimal
  Termination message: MIP - Integer optimal, tolerance (0.0001/1e-06)\x3a Objective = 1.4596344360e+06
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 14054
      Number of created subproblems: 14054
  Error rc: 0
  Time: 2283.7451367378235
Solution: 
- number of solutions: 0
  number of solutions displayed: 0



In [84]:
P_sell = np.zeros((len(sellPrice),1))
for i in m.Time:
    P_sell[i] = en.value(m.P_sell[i])

In [85]:
P_buy = np.zeros((len(sellPrice),1))
for i in m.Time:
    P_buy[i] = en.value(m.P_buy[i])

In [86]:
P_PV = np.zeros((len(sellPrice),1))
for i in m.Time:
    P_PV[i] = en.value(m.P_PV[i])

In [87]:
n_PV = en.value(m.n_PV[1])
print(n_PV)

6493.0


In [88]:
n_HP = en.value(m.n_HP[1])
print(n_HP)

8.0


In [89]:
n_GT = en.value(m.n_GT[1])
print(n_GT)

7.0


In [90]:
n_CHP = en.value(m.n_CHP[1])
print(n_CHP)

3.0


In [91]:
n_EC = en.value(m.n_EC[1])
print(n_EC)

1.0


In [92]:
n_AC = en.value(m.n_AC[1])
print(n_AC)

4.0


In [93]:
n_ESS = en.value(m.n_ESS[1])
print(n_ESS)

0.0


In [94]:
n_TSS = en.value(m.n_TSS[1])
print(n_TSS)

45.0


In [95]:
Q_AC = np.zeros((len(sellPrice),1))
for i in m.Time:
    Q_AC[i] = en.value(m.Q_AC[i])

In [96]:
P_GT = np.zeros((len(sellPrice),1))
for i in m.Time:
    P_GT[i] = en.value(m.P_GT[i])

In [97]:
P_CHP = np.zeros((len(sellPrice),1))
for i in m.Time:
    P_CHP[i] = en.value(m.P_CHP[i])

In [98]:
Q_CHP = np.zeros((len(sellPrice),1))
for i in m.Time:
    Q_CHP[i] = en.value(m.Q_CHP[i])

In [99]:
P_HP = np.zeros((len(sellPrice),1))
for i in m.Time:
    P_HP[i] = en.value(m.P_HP[i])

In [100]:
P_EC = np.zeros((len(sellPrice),1))
for i in m.Time:
    P_EC[i] = en.value(m.P_EC[i])

In [101]:
TSS_P_dis = np.zeros((len(sellPrice),1))
for i in m.Time:
    TSS_P_dis[i] = en.value(m.TSS_P_dis[i])

In [102]:
TSS_P_char = np.zeros((len(sellPrice),1))
for i in m.Time:
    TSS_P_char[i] = en.value(m.TSS_P_char[i])

In [103]:
Q_EC = P_EC*4

In [104]:
Q_HP = P_HP*2

In [105]:
# Output the cost of purchasing and selling electricity
C_grid = sum( (buyPrice[i]*P_buy[i]) - (sellPrice[i]*P_sell[i]) for i in range(8760) )

C_grid_heating1 = sum( (buyPrice[i]*P_buy[i]) - (sellPrice[i]*P_sell[i]) for i in range(0,3624) ) 
C_grid_heating2 = sum( (buyPrice[i]*P_buy[i]) - (sellPrice[i]*P_sell[i]) for i in range(6552,8760) ) 
C_grid_heating = C_grid_heating1 + C_grid_heating2

C_grid_cooling = sum( (buyPrice[i]*P_buy[i]) - (sellPrice[i]*P_sell[i]) for i in range(3624,6552) ) 

print(C_grid)

[164108.01728629]


In [106]:
print(C_grid_heating)

[119020.46124156]


In [107]:
print(C_grid_cooling)

[45087.55604473]


In [108]:
# Output investment cost
C_inv = R25*198*n_PV + R25*70572*n_HP + R25*203808*n_GT + R25*505766*n_CHP + R25*36438*n_EC + R25*27483*n_AC + R10*444*n_ESS + R20*1400*n_TSS        
print(C_inv)

469238.00856501365


In [109]:
# Output the cost of purchasing gas
Vgas_GT = np.zeros((len(sellPrice),1))
for i in m.Time:
    Vgas_GT[i] = en.value(m.Vgas_GT[i])
    
Vgas_CHP = np.zeros((len(sellPrice),1))
for i in m.Time:
    Vgas_CHP[i] = en.value(m.Vgas_CHP[i])


C_gas = sum( (price_gas[i]*(Vgas_GT[i]+Vgas_CHP[i])) for i in range(8760))

C_gas_heating1 = sum( (price_gas[i]*(Vgas_GT[i]+Vgas_CHP[i])) for i in range(0,3624))
C_gas_heating2 = sum( (price_gas[i]*(Vgas_GT[i]+Vgas_CHP[i])) for i in range(6552,8760))
C_gas_heating = C_gas_heating1 + C_gas_heating2

C_gas_cooling = sum( (price_gas[i]*(Vgas_GT[i]+Vgas_CHP[i])) for i in range(3624,6552)) 

print(C_gas)

[643030.56830198]


In [110]:
print(C_gas_heating)

[546741.01072644]


In [111]:
print(C_gas_cooling)

[96289.55757554]


In [112]:
# Output Maintenance cost
C_maintain = 0.03*198*n_PV + 0.03*70572*n_HP + 0.03*203808*n_GT + 0.03*505766*n_CHP + 0.03*36438*n_EC + 0.03*27483*n_AC + 0.03*444*n_ESS + 0.03*1400*n_TSS    
print(C_maintain)

150105.42


In [113]:
# Output CO2 emission penalizes the cost
C_emission = sum( (1*5*0.00089*P_buy[i]) + (1*5*0.0001862*9.97*(Vgas_GT[i]+Vgas_CHP[i])) for i in range(8760))

C_emission_heating1=sum( (1*5*0.00089*P_buy[i])+(1*5*0.0001862*9.97*(Vgas_GT[i]+Vgas_CHP[i])) for i in range(0,3624))
C_emission_heating2=sum( (1*5*0.00089*P_buy[i])+(1*5*0.0001862*9.97*(Vgas_GT[i]+Vgas_CHP[i])) for i in range(6552,8760))
C_emission_heating = C_emission_heating1 + C_emission_heating2

C_emission_cooling = sum( (1*5*0.00089*P_buy[i]) + (1*5*0.0001862*9.97*(Vgas_GT[i]+Vgas_CHP[i])) for i in range(3625,6552))

print(C_emission)

[33152.4218583]


In [114]:
print(C_emission_heating)

[26598.28327121]


In [115]:
print(C_emission_cooling)

[6551.60005649]


In [116]:
# Output annualized total cost
C_total = C_emission + C_maintain + C_gas + C_inv + C_grid
print(C_total)

[1459634.43601158]
